This notebook is used for randomly sampling from the 

In [ ]:
import numpy as np

import glob

import os

import pandas as pd

import shutil

from zipfile import ZipFile

In [ ]:
DATADIR = './data/pericardial/wsx_round2/'

nameMapFile = './data/TT_804_32k_bridge_by_feid.csv'#this file contains a mapping between patient names within UK Biobank and the f.eid which were somehow stripped from dicoms because I messed up

nameMap = pd.read_csv(nameMapFile).set_index('Patient_name')

In [ ]:
#get a list of all of the wsx files
allwsx = glob.glob(os.path.join(DATADIR,'*.cvi42wsx'))

First - a random subset of 44 for 

In [ ]:
#determinism yo
np.random.seed(11811)

sampleSize = np.ceil(len(allwsx) / 10).astype(int)

#pick a subsample
randomSample = np.random.choice(allwsx,size = sampleSize,replace=False)

#create a subfolder
SAMPLEDIR = os.path.join(DATADIR,'QC_sample')

if not os.path.isdir(SAMPLEDIR):
    os.mkdir(SAMPLEDIR)

#copy all the files across
[shutil.copy(x,SAMPLEDIR) for x in randomSample]

#remove the filepaths from the names...
randomSample = [os.path.basename(x) for x in randomSample]

#get the patient names
patientNames = [x[:8] for x in randomSample]

#get the f.eids using the lookup table..
feids = nameMap.loc[patientNames,'f.eid']

#create a DataFrame and write the 
dataFrame = pd.DataFrame({'f.eid':feids,'filename':randomSample}).sort_index()
dataFrame.to_csv(os.path.join(SAMPLEDIR,'QC_random_sample.csv'))


allFiles = glob.glob(SAMPLEDIR)

#zip up the subsample directory

with ZipFile(SAMPLEDIR + '.zip', 'w') as zipObj:
   # Iterate over all the files in directory
   for folderName, subfolders, filenames in os.walk(SAMPLEDIR):
        for filename in filenames:
            #create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            # Add file to zip
            zipObj.write(filePath)
